# Scratch-like Programming in Haskell

## Interactive Guide

This notebook demonstrates Scratch-like programming concepts using pure functional Haskell.

### Ways to Use This Notebook:

1. **Run in Desktop iHaskell** - Execute each cell to see how the code works
2. **Run in xeus-iHaskell** - Execute each cell to see how the code works
3. **Copy to xeus-iHaskell MicroHS Console** - Copy the code from cells to paste into your console (see scratch_microhs_guide.pdf)

### Important Notes:

- **'st' represents a GameState value** (the game configuration)
- **GameState is immutable** - we create new values, never modify existing ones
- **All functions create NEW GameStates** - the original is unchanged

## Section 1: Define GameState

GameState represents the current game configuration with position and score.

In [1]:
-- Define the GameState type with three fields: xPos, yPos, score
data GameState = GameState { xPos :: Int, yPos :: Int, score :: Int } deriving Show

In [2]:
-- Create the initial GameState (all zeros)
initialState = GameState 0 0 0
initialState

GameState {xPos = 0, yPos = 0, score = 0}

## Section 2: Basic Movement Functions

These functions create NEW GameStates with different positions.

In [3]:
-- gotoXY: moves to position (x, y)
-- st :: GameState (current game configuration)
-- Returns: NEW GameState with position set to (x, y)
gotoXY x y st = st { xPos = x, yPos = y }

In [4]:
:type gotoXY

gotoXY :: Int -> Int -> GameState -> GameState

In [5]:
-- changeScoreBy: adds delta to the score
-- st :: GameState (current game configuration)
-- Returns: NEW GameState with score increased by delta
changeScoreBy delta st = st { score = score st + delta }

In [6]:
:type changeScoreBy

changeScoreBy :: Int -> GameState -> GameState

In [7]:
-- setScore: sets score to a specific value
-- st :: GameState (current game configuration)
-- Returns: NEW GameState with score set to value
setScore value st = st { score = value }

In [8]:
:type setScore

setScore :: Int -> GameState -> GameState

In [9]:
-- moveSteps: moves by 'steps' in x direction
-- st :: GameState (current game configuration)
-- Returns: NEW GameState with x position changed by steps
moveSteps steps st = st { xPos = xPos st + steps }

In [10]:
-- turnRight: moves 10 steps to the right
turnRight st = moveSteps 10 st

-- turnLeft: moves 10 steps to the left
turnLeft st = moveSteps (-10) st

-- moveUp: moves 10 steps up
moveUp st = st { yPos = yPos st + 10 }

-- moveDown: moves 10 steps down
moveDown st = st { yPos = yPos st - 10 }

Line 2: Eta reduce
Found:
turnRight st = moveSteps 10 st
Why not:
turnRight = moveSteps 10Line 5: Eta reduce
Found:
turnLeft st = moveSteps (- 10) st
Why not:
turnLeft = moveSteps (- 10)

### Test Movement Functions

In [11]:
-- Test: move right from initial position
turnRight initialState

GameState {xPos = 10, yPos = 0, score = 0}

In [12]:
-- Test: move to a specific position
gotoXY 50 50 initialState

GameState {xPos = 50, yPos = 50, score = 0}

In [13]:
-- Test: chain movements
moveUp (turnRight initialState)

GameState {xPos = 10, yPos = 10, score = 0}

## Section 3: Control Structures

These functions implement Scratch-like control flow.

**Note:** These use pattern matching and guards for cleaner, more idiomatic Haskell code.

In [14]:
-- repeatN: repeat function f exactly n times
-- n :: Int (number of times to repeat)
-- f :: (a -> a) (function to apply)
-- x :: a (starting value)
-- Returns: final value after n applications
repeatN 0 f x = x
repeatN n f x = repeatN (n-1) f (f x)

In [15]:
:type repeatN

repeatN :: forall {t1} {t2}. (Eq t1, Num t1) => t1 -> (t2 -> t2) -> t2 -> t2

In [16]:
-- Test: move right 5 times
repeatN 5 turnRight initialState

GameState {xPos = 50, yPos = 0, score = 0}

In [17]:
-- repeatUntil: repeat until condition is true
-- cond :: (a -> Bool) (condition to check)
-- action :: (a -> a) (function to apply)
-- maxIters :: Int (safety limit)
-- x :: a (current value)
-- Returns: value when condition becomes true
repeatUntil cond action 0 x = x
repeatUntil cond action maxIters x
  | cond x = x
  | otherwise = repeatUntil cond action (maxIters - 1) (action x)

In [18]:
:type repeatUntil

repeatUntil :: forall {t1} {t2}. (Eq t1, Num t1) => (t2 -> Bool) -> (t2 -> t2) -> t1 -> t2 -> t2

In [19]:
-- Test: move right until x >= 30
-- Lambda \st means: st :: GameState
repeatUntil (\st -> xPos st >= 30) turnRight 100 initialState

GameState {xPos = 30, yPos = 0, score = 0}

In [20]:
-- ifThen: conditionally apply action
ifThen condition action st = if condition then action st else st

In [21]:
:type ifThen

ifThen :: forall {t}. Bool -> (t -> t) -> t -> t

## Section 4: Display Functions

Helper functions to show GameState in a readable format.

In [22]:
-- displayState: show GameState as readable string
-- st :: GameState (the GameState to display)
-- Returns: String describing the position and score
displayState st = "Position: (" ++ show (xPos st) ++ ", " ++ show (yPos st) ++ "), Score: " ++ show (score st)

In [23]:
-- Test: display initial state
putStrLn $ displayState initialState

Position: (0, 0), Score: 0

In [24]:
-- Test: display after movement
putStrLn $ displayState (turnRight initialState)

Position: (10, 0), Score: 0

In [25]:
-- executeCommands: apply a list of commands in sequence
-- [GameState -> GameState] (list of commands)
-- st :: GameState (starting GameState)
-- Returns: final NEW GameState after all commands
executeCommands [] st = st
executeCommands (cmd:cmds) st = executeCommands cmds (cmd st)

Line 5: Use foldl
Found:
executeCommands [] st = st
executeCommands (cmd : cmds) st = executeCommands cmds (cmd st)
Why not:
executeCommands cmds st = foldl (\ st cmd -> cmd st) st cmds

## Section 5: Demo Programs

Complete examples demonstrating different patterns.

### Demo 1: Move in a Square

In [26]:
-- demo1: move in a square pattern
-- Sequence: (0,0) -> (50,0) -> (50,50) -> (0,50) -> (0,0)
demo1 = gotoXY 0 0 (gotoXY 0 50 (gotoXY 50 50 (gotoXY 50 0 (gotoXY 0 0 initialState))))

putStrLn $ displayState demo1

Position: (0, 0), Score: 0

### Demo 2: Repeat Movement

In [27]:
-- demo2: move right 5 times
-- Result: x = 50, y = 0, score = 0
demo2 = repeatN 5 turnRight initialState

putStrLn $ displayState demo2

Position: (50, 0), Score: 0

### Demo 3: Repeat Until Score = 10

In [28]:
-- demo3: increase score until it reaches 10
demo3 = repeatUntil (\st -> score st >= 10) (changeScoreBy 1) 100 initialState

putStrLn $ displayState demo3

Position: (0, 0), Score: 10

### Demo 4: Repeat Until X >= 50

In [29]:
-- demo4: move right until x position >= 50
demo4 = repeatUntil (\st -> xPos st >= 50) turnRight 100 initialState

putStrLn $ displayState demo4

Position: (50, 0), Score: 0

### Demo 5: Complex Pattern

In [30]:
-- demo5: complex pattern
-- 1. Set score to 0
-- 2. Move right 3 times (x=30)
-- 3. Move up 2 times (y=20)
-- 4. Add 10 to score (score=10)
-- 5. Return to origin (0,0)
demo5 = gotoXY 0 0 (changeScoreBy 10 (repeatN 2 moveUp (repeatN 3 turnRight (setScore 0 initialState))))

putStrLn $ displayState demo5

Position: (0, 0), Score: 10

## Section 6: Command Sequences

Execute a list of commands in order.

In [31]:
-- sequence1: a list of commands to execute
-- Type: [GameState -> GameState]
sequence1 = [turnRight, turnRight, moveUp, changeScoreBy 5, turnLeft]

-- commandDemo: execute the sequence
commandDemo = executeCommands sequence1 initialState

putStrLn $ displayState commandDemo

Position: (10, 10), Score: 5

## Section 7: Pseudo-Random (Optional)

Simple pseudo-random number generation without external libraries.

In [32]:
-- Seed: type alias for random number generator configuration
type Seed = Int

In [33]:
-- nextRandom: generate next random number and seed
-- seed :: Int (current seed)
-- Returns: (newSeed, randomValue) where randomValue is 0-99
nextRandom seed = 
  let newSeed = (1103515245 * seed + 12345) `mod` 2147483648
      value = newSeed `mod` 100
  in (newSeed, value)

In [34]:
-- pickRandomInRange: generate random in range [low, high]
-- low :: Int, high :: Int (range bounds)
-- seed :: Seed (random generator configuration)
-- Returns: (randomValue, newSeed)
pickRandomInRange low high seed = 
  let (newSeed, value) = nextRandom seed
      rangeSize = high - low + 1
      result = low + (value `mod` rangeSize)
  in (result, newSeed)

In [35]:
-- Test: generate random number between 1 and 10
pickRandomInRange 1 10 42

(8,1250496027)

In [36]:
-- Test: generate multiple random numbers with different seeds
let (val1, seed1) = pickRandomInRange 1 6 42
let (val2, seed2) = pickRandomInRange 1 6 seed1
let (val3, seed3) = pickRandomInRange 1 6 seed2
print [val1, val2, val3]

[4,5,6]

## Section 8: Game With Seed (Optional)

Combine GameState with random number generation for dice rolling.

In [37]:
-- GameWithSeed: combines GameState with random seed
data GameWithSeed = GameWithSeed 
  { gameState :: GameState
  , seed :: Seed 
  } deriving Show

In [38]:
-- rollAndScore: roll dice (1-6) and add to score
-- gws :: GameWithSeed (game configuration with seed)
-- Returns: COMPLETELY NEW GameWithSeed with new score and new seed
rollAndScore gws = 
  let (roll, newSeed) = pickRandomInRange 1 6 (seed gws)
      newScore = score (gameState gws) + roll
      newGameState = (gameState gws) { score = newScore }
  in GameWithSeed newGameState newSeed

In [39]:
-- playGame: roll dice 5 times
-- initialSeed :: Int (starting random seed)
-- Returns: final GameWithSeed after 5 rolls
playGame initialSeed = 
  repeatN 5 rollAndScore (GameWithSeed initialState initialSeed)

In [40]:
-- Test: play game with seed 42
playGame 42

GameWithSeed {gameState = GameState {xPos = 0, yPos = 0, score = 22}, seed = 908095735}

In [41]:
-- Test: play game with different seed
playGame 123

GameWithSeed {gameState = GameState {xPos = 0, yPos = 0, score = 19}, seed = 1143491652}

In [42]:
-- Test: display final game state
let result = playGame 42
putStrLn $ displayState (gameState result)

Position: (0, 0), Score: 22

## Section 9: Animation Trace (Optional)

Record all intermediate states during execution.

In [43]:
-- traceSteps: record all intermediate GameStates
-- f :: (a -> a) (function to apply)
-- n :: Int (number of steps)
-- x :: a (starting value)
-- Returns: list of all values [x, f(x), f(f(x)), ...]
traceSteps f 0 x = [x]
traceSteps f n x = x : traceSteps f (n-1) (f x)

In [44]:
-- animateDemo: trace 5 steps of turnRight
animateDemo = traceSteps turnRight 5 initialState

-- Show each step
mapM_ (putStrLn . displayState) animateDemo

Position: (0, 0), Score: 0
Position: (10, 0), Score: 0
Position: (20, 0), Score: 0
Position: (30, 0), Score: 0
Position: (40, 0), Score: 0
Position: (50, 0), Score: 0

## Understanding 'st'

### What is 'st'?

'st' is a parameter that represents a GameState value. Think of it as the current game configuration.

### Important: Two Different Uses of 'st'

**Example 1: Parameter only used in record update (gotoXY)**

```haskell
gotoXY x y st = st { xPos = x, yPos = y }
           ^^   ^^
           |    This is the BASE RECORD for record update syntax
           |
           This declares the parameter (only used once)
```

**Example 2: Parameter used multiple times (moveUp)**

```haskell
moveUp st = st { yPos = yPos st + 10 }
       ^^   ^^         ^^^^^  ^^
       |    |          |      Parameter st referenced here!
       |    |          Field accessor function
       |    Base record for update syntax
       Parameter declaration
```

In `moveUp`, the parameter `st` appears **THREE times**:
1. **Left of `=`** - declares the parameter
2. **Before `{`** - the base record for update syntax
3. **In `yPos st`** - explicitly referenced to get the current yPos value

### Record Update Syntax Explained

In `st { xPos = x, yPos = y }`:

- **`st`** before the `{` is the **base record**
- It supplies:
  - The constructor (`GameState`)
  - All other fields unchanged (`score` stays the same)
- The `{ xPos = x, yPos = y }` part **overrides** just those fields
- Result: A COMPLETELY NEW GameState with updated fields

### Example 1: Parameter NOT explicitly referenced (gotoXY)

```haskell
gotoXY x y st = st { xPos = x, yPos = y }
```

Here, the parameter `st` on the left is used ONLY in the record update syntax.
It's not referenced elsewhere in the definition.

### Example 2: Parameter IS explicitly referenced (moveUp)

```haskell
moveUp st = st { yPos = yPos st + 10 }
                        ^^^^^  ^^
                        |      Parameter st explicitly referenced
                        Field accessor function
```

Here, `st` appears THREE times:
1. **Left of `=`** - declares the parameter
2. **Before `{`** - the base record for update syntax
3. **In `yPos st`** - explicitly referenced to get current yPos value

### Key Insight: Record Update

```haskell
st { field = newValue }
```

This means:
- Take the record `st`
- Create a BRAND NEW record of the same type
- Copy all fields from `st`
- Override the specified field(s) with new value(s)
- The original `st` is NEVER modified (immutable)

### IMPORTANT: IMMUTABILITY

- 'st' itself is NEVER changed or modified
- Record update creates a COMPLETELY NEW GameState
- The original 'st' remains exactly as it was
- We create new values, we never modify existing ones

### More Examples

```haskell
-- Only uses st in record update (not explicitly referenced)
setScore value st = st { score = value }

-- Explicitly references st to read current score
changeScoreBy delta st = st { score = score st + delta }
                                      ^^^^^  ^^
                                      accessor + parameter
```

## Practice Examples

### Example 1: Simple Movement

In [45]:
turnRight initialState

GameState {xPos = 10, yPos = 0, score = 0}

In [46]:
moveUp (turnRight initialState)

GameState {xPos = 10, yPos = 10, score = 0}

In [47]:
putStrLn $ displayState (moveUp (turnRight initialState))

Position: (10, 10), Score: 0

### Example 2: Repeat Patterns

In [48]:
repeatN 3 turnRight initialState

GameState {xPos = 30, yPos = 0, score = 0}

In [49]:
putStrLn $ displayState (repeatN 3 turnRight initialState)

Position: (30, 0), Score: 0

### Example 3: Until Conditions

In [50]:
-- Move right until score >= 5 (increase score each time)
repeatUntil (\s -> score s >= 5) (changeScoreBy 1) 100 initialState

GameState {xPos = 0, yPos = 0, score = 5}

In [51]:
putStrLn $ displayState (repeatUntil (\s -> score s >= 5) (changeScoreBy 1) 100 initialState)

Position: (0, 0), Score: 5

### Example 4: Function Composition

In [52]:
-- Compose functions: first turnRight, then moveUp
(moveUp . turnRight) initialState

GameState {xPos = 10, yPos = 10, score = 0}

In [53]:
putStrLn $ displayState ((moveUp . turnRight) initialState)

Position: (10, 10), Score: 0

### Example 5: Chaining Operations

In [54]:
moveUp (moveUp (turnRight (turnRight initialState)))

GameState {xPos = 20, yPos = 20, score = 0}

### Example 6: Custom Sequences

In [55]:
myCommands = [turnRight, moveUp, changeScoreBy 10]
result = executeCommands myCommands initialState
putStrLn $ displayState result

Position: (10, 10), Score: 10

### Example 7: Building Step by Step

In [56]:
s1 = repeatN 3 turnRight initialState
s2 = moveUp s1
s3 = changeScoreBy 5 s2
putStrLn $ displayState s3

Position: (30, 10), Score: 5

## Summary

### Key Concepts:

1. **'st' is a GameState value**
   - It represents the current game configuration
   - It has three fields: xPos, yPos, score

2. **Functions CREATE NEW GameStates**
   - They take a GameState (st) as input
   - They return a COMPLETELY NEW GameState as output
   - The original GameState is NEVER changed (immutable)

3. **Use composition to chain functions**
   - `(f . g) x` means: apply g, then apply f
   - `(moveUp . turnRight) initialState`
   - This creates new GameState from turnRight, then new GameState from moveUp

4. **GameState is IMMUTABLE**
   - We NEVER modify existing GameStates
   - `st { xPos = 10 }` creates a BRAND NEW GameState
   - The original 'st' remains exactly as it was
   - Each function creates a fresh new value

5. **'st' is just a naming convention**
   - You could use any name: st, s, game, g
   - We use 'st' consistently for clarity
   - It always represents a GameState value
   - It is never modified, only used as a template

## For MicroHS Console Users

To use this code in the MicroHS console:

1. Copy the code from each cell (one at a time)
2. Paste into your MicroHS console
3. Press Enter
4. Test with the examples provided

### Important Notes for Console:

- **Pattern matching may not work in console** - If you get errors with functions like `repeatN`, try converting to `if-then-else`:
  - Instead of: `repeatN 0 f x = x; repeatN n f x = ...`
  - Use: `repeatN n f x = if n == 0 then x else ...`
- Copy ONE definition at a time
- For pattern-matched functions, copy all patterns together on one line with semicolons
- Test each function before moving to the next section

### Example Console Conversion:

If `repeatN 0 f x = x` followed by `repeatN n f x = repeatN (n-1) f (f x)` doesn't work, use:

```haskell
repeatN n f x = if n == 0 then x else repeatN (n-1) f (f x)
```